In [102]:
%load_ext autoreload
%autoreload 2
from building_dection import *

import os
import rasterio as rio
import numpy as np
import json
import pyproj
from osgeo import gdal
import subprocess


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
labels = "/scratch/toml/sn7/qgis/labels2.geojson"

vector_ds = gdal.OpenEx(labels, gdal.OF_VECTOR)
vector_extent = vector_ds.GetLayer().GetExtent()

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date

user, password = get_credentials()
api = SentinelAPI(user, password, "https://scihub.copernicus.eu/dhus")

footprint = f"POLYGON(({vector_extent[0]} {vector_extent[2]}, {vector_extent[1]} {vector_extent[2]}, {vector_extent[1]} {vector_extent[3]}, {vector_extent[0]} {vector_extent[3]}, {vector_extent[0]} {vector_extent[2]}))"

products = api.query(
    footprint,
    date=("20230801", "20231031"),
    platformname="Sentinel-2",
    processinglevel="Level-1C",
    cloudcoverpercentage=(0, 30),
)

products_df = api.to_dataframe(products)

In [122]:
for prod in products:
    is_online = api.is_online(prod)
    print(f"{prod}: {is_online}")

b09f1a11-bfa5-4993-98aa-125d9ea9da10: True
f10ff1b4-9879-436a-aae9-1d957074f6dc: True
82d5e839-9c67-486c-af0a-a4c23abfcf13: True
94867d2d-9c27-4059-b471-a5aefb095ef6: True
ba1cf43f-172d-44a6-a862-30dfbe18ea1d: True
727592ed-a56c-4e63-8466-a0dabac74a2f: True
371fba26-aaa8-4048-948c-e3c85aa36fd8: False
0929c05e-72ab-42a7-8bc7-3eef96e4239f: False
94b32aca-6782-4047-9d6f-b80846f1369f: False
a3ac6b68-7c15-4375-adb4-0acf58979496: False
fdb3b9ce-73f6-488c-9ed0-d41a48365be5: False
30227edb-1ca1-4b25-89b5-6ed049e7c061: False
a7d98e81-1a26-41da-a1ab-f785267bd26d: False
85177817-1565-4340-8ae4-5467e960d4e2: False


In [125]:
api.download("b09f1a11-bfa5-4993-98aa-125d9ea9da10", directory_path="/scratch/toml/sn7/sentinel2")

{'id': 'b09f1a11-bfa5-4993-98aa-125d9ea9da10',
 'title': 'S2B_MSIL1C_20231011T185319_N0509_R113_T10SFH_20231011T204156',
 'size': 843428749,
 'md5': 'e2249770092018e948b2eb1575518c3a',
 'date': datetime.datetime(2023, 10, 11, 18, 53, 19, 24000),
 'footprint': 'POLYGON((-121.8477 38.84331460585068,-120.58327 38.82399234658653,-120.615906 37.83530211029287,-121.86328 37.853955038178526,-121.8477 38.84331460585068))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('b09f1a11-bfa5-4993-98aa-125d9ea9da10')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2023, 10, 11, 22, 22, 0, 185000),
 'Ingestion Date': datetime.datetime(2023, 10, 11, 22, 20, 57, 10000),
 'manifest_name': 'manifest.safe',
 'product_root_dir': 'S2B_MSIL1C_20231011T185319_N0509_R113_T10SFH_20231011T204156.SAFE',
 'quicklook_url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('b09f1a11-bfa5-4993-98aa-125d9ea9da10')/Products('Quicklook')/$value",
 'path': '/scratch/toml/sn7/sentinel2/S2B_MSIL

In [ ]:
# Define the EPSG codes for the source and target coordinate systems
src_crs = 'EPSG:4326'
tgt_crs = 'EPSG:32610'  # UTM zone 10N in WGS84

labels = '/scratch/toml/sn7/qgis/labels2.geojson'
reproj_labels = '/scratch/toml/sn7/qgis/labels2_reproj.geojson'
raster_labels = '/scratch/toml/sn7/qgis/labels2.tif'
extent = [600000.0, 4190220.0, 709800.0, 4300020.0]

# Reproject the GeoJSON to the target coordinate system
reproject_geojson(labels, src_crs, tgt_crs, out_fn=reproj_labels)
rasterize_labels(reproj_labels, raster_labels, extent)


In [101]:

l1c = '/scratch/toml/sn7/qgis/l1c_set1 (copy).tif'
l1c_crop = '/scratch/toml/sn7/qgis/l1c_set1_crop.tif'
raster_labels = '/scratch/toml/sn7/qgis/labels2.tif'
raster_labels_crop = '/scratch/toml/sn7/qgis/labels2_crop.tif'
crop_to_extent(labels, l1c, l1c_crop)
crop_to_extent(labels, raster_labels, raster_labels_crop)


GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AXIS["Latitude",NORTH],
    AXIS["Longitude",EAST],
    AUTHORITY["EPSG","4326"]]
(-121.68466515096087, -121.64058492652846, 37.9615752659464, 37.99617689154867)
PROJCS["WGS 84 / UTM zone 10N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-123],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Eas